In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
#explore the tables to understand what columns exits


# Exploratory Precipitation Analysis

In [ ]:
#understand what columns are in both tables
row = session.query(measurement).first()
row.__dict__

In [ ]:
#understand the columns for the second table
second_row = session.query(station).first()
second_row.__dict__

In [ ]:
# Find the most recent date in the data set.
recent_date = session.query(measurement.date).order_by(measurement.date.desc()).first()
recent_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
date = dt.date(2017,8,23) - dt.timedelta(days=365)
print("Query Date: ", date)

In [ ]:
# Perform a query to retrieve the data and precipitation scores
precip_data = session.query(measurement.date, measurement.prcp).filter(measurement.date >= date).all()
for row in precip_data:
    print(row)

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
precip_df = pd.DataFrame(precip_data, columns = ['date', "prcp"])
precip_df.set_index("date", inplace=True)
precip_df

In [ ]:
# Sort the dataframe by date
precip_df = precip_df.sort_values('date')
precip_df

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
plt.figure(figsize=(12,9))
plt.plot(precip_df.index, precip_df['prcp'])

xloc = ['2016-08-23', '2016-10-11', '2016-11-09', '2016-12-19', '2017-01-28', '2017-03-09', '2017-04-18','2017-05-31','2017-07-10']
# rain_plot = precip_df.plot(rot = 45, figsize=(9,6), title = "Hawaii Rainfall")
plt.xlabel('Date')
plt.ylabel('Inches')
plt.title("Hawaii Rainfall")
plt.xticks(xloc, rotation="vertical")
plt.legend(["Precipitation"], fontsize=10)

plt.tight_layout()
plt.savefig("Images/Precip_Plot.png")
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
precip_df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
session.query(station).group_by(station.station).count()

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
session.query(measurement.station, func.count(measurement.date)).group_by(measurement.station).\
order_by(func.count(measurement.station).desc()).all()

In [ ]:
# isolate the highest id
active_station = session.query(measurement.station).group_by(measurement.station).\
    order_by(func.count(measurement.date).desc()).first()
most_active = active_station[0]
most_active

In [ ]:
#calcuclate the the lowest, highest, and average temperature based on the id
session.query(measurement.station, func.min(measurement.tobs), func.max(measurement.tobs), func.avg(measurement.tobs)).\
    filter(measurement.station==most_active).all()

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
data = session.query(measurement.tobs).filter(measurement.station==most_active).filter(measurement.date>=date).all()
data

In [ ]:
plt.figure(figsize=(15,10))
plt.hist(np.ravel(data))
plt.title("Temperature Observation Data")
plt.xlabel("Temperature")
plt.ylabel("Frequency")
plt.legend(["Tobs"], fontsize = 20)

plt.savefig("Images/TOBS_Histogram.png")
plt.tight_layout
plt.show()

# Close session

In [ ]:
# Close Session
session.close()